### Install the Library

In [1]:
import requests
import pandas as pd

### SEC API

In [2]:
# create request header
headers = {'User-Agent': "aariyur@berkeley.edu"}

In [3]:
companyTickers = requests.get(
    "https://www.sec.gov/files/company_tickers.json",
    headers=headers
)

# review response / keys
print(companyTickers.json())

{'0': {'cik_str': 320193, 'ticker': 'AAPL', 'title': 'Apple Inc.'}, '1': {'cik_str': 789019, 'ticker': 'MSFT', 'title': 'MICROSOFT CORP'}, '2': {'cik_str': 1652044, 'ticker': 'GOOGL', 'title': 'Alphabet Inc.'}, '3': {'cik_str': 1018724, 'ticker': 'AMZN', 'title': 'AMAZON COM INC'}, '4': {'cik_str': 1067983, 'ticker': 'BRK-B', 'title': 'BERKSHIRE HATHAWAY INC'}, '5': {'cik_str': 1318605, 'ticker': 'TSLA', 'title': 'Tesla, Inc.'}, '6': {'cik_str': 1045810, 'ticker': 'NVDA', 'title': 'NVIDIA CORP'}, '7': {'cik_str': 1046179, 'ticker': 'TSM', 'title': 'TAIWAN SEMICONDUCTOR MANUFACTURING CO LTD'}, '8': {'cik_str': 1403161, 'ticker': 'V', 'title': 'VISA INC.'}, '9': {'cik_str': 1326801, 'ticker': 'META', 'title': 'Meta Platforms, Inc.'}, '10': {'cik_str': 34088, 'ticker': 'XOM', 'title': 'EXXON MOBIL CORP'}, '11': {'cik_str': 731766, 'ticker': 'UNH', 'title': 'UNITEDHEALTH GROUP INC'}, '12': {'cik_str': 824046, 'ticker': 'LVMUY', 'title': 'LVMH MOET HENNESSY LOUIS VUITTON'}, '13': {'cik_str'

In [4]:
# parse CIK // without leading zeros
directCik = companyTickers.json()['0']['cik_str']


# dictionary to dataframe
companyData = pd.DataFrame.from_dict(companyTickers.json(),
                                     orient='index')
# add leading zeros to CIK such that it matches the format of 10 digits
companyData['cik_str'] = companyData['cik_str'].astype(
                           str).str.zfill(10)

In [5]:
companyData.head()

,cik_str,ticker,title
0,0000320193,AAPL,Apple Inc.
1,0000789019,MSFT,MICROSOFT CORP
2,0001652044,GOOGL,Alphabet Inc.
3,0001018724,AMZN,AMAZON COM INC
4,0001067983,BRK-B,BERKSHIRE HATHAWAY INC


In [21]:
import json
combined_metadata = []

for i in range(5):
    cik = companyData.cik_str[i]
    # get company specific filing metadata
    filingMetadata = requests.get(
        f'https://data.sec.gov/submissions/CIK{cik}.json',
        headers=headers
    )
    # append the JSON response to the list
    combined_metadata.append(json.loads(filingMetadata.content))
    
# convert the list to a JSON string
combined_metadata_json = json.dumps(combined_metadata)
print(combined_metadata_json)


[{"cik": "320193", "entityType": "operating", "sic": "3571", "sicDescription": "Electronic Computers", "insiderTransactionForOwnerExists": 0, "insiderTransactionForIssuerExists": 1, "name": "Apple Inc.", "tickers": ["AAPL"], "exchanges": ["Nasdaq"], "ein": "942404110", "description": "", "website": "", "investorWebsite": "", "category": "Large accelerated filer", "fiscalYearEnd": "0930", "stateOfIncorporation": "CA", "stateOfIncorporationDescription": "CA", "addresses": {"mailing": {"street1": "ONE APPLE PARK WAY", "street2": null, "city": "CUPERTINO", "stateOrCountry": "CA", "zipCode": "95014", "stateOrCountryDescription": "CA"}, "business": {"street1": "ONE APPLE PARK WAY", "street2": null, "city": "CUPERTINO", "stateOrCountry": "CA", "zipCode": "95014", "stateOrCountryDescription": "CA"}}, "phone": "(408) 996-1010", "flags": "", "formerNames": [{"name": "APPLE INC", "from": "2007-01-10T00:00:00.000Z", "to": "2019-08-05T00:00:00.000Z"}, {"name": "APPLE COMPUTER INC", "from": "1994-01

## Can use similar techniques as in the next two cells to get json and xml data for other companies

In [31]:
import json

# assume combined_metadata is a list of dictionaries
with open('filing_metadata.json', 'w') as outfile:
    json.dump(filingMetadata.json(), outfile)


In [35]:
import sys
!pip install xmltodict

Defaulting to user installation because normal site-packages is not writeable


## Data Analysis for One Company

In [22]:
allForms = pd.DataFrame.from_dict(
             filingMetadata.json()['filings']['recent']
           )

In [36]:

import xmltodict
# convert JSON data to XML
xml_data = xmltodict.unparse({'metadata': {'filing': filingMetadata.json()}})

# write XML data to file
with open('filing_metadata.xml', 'w') as outfile:
    outfile.write(xml_data)


In [23]:
allForms.columns

Index(['accessionNumber', 'filingDate', 'reportDate', 'acceptanceDateTime',
       'act', 'form', 'fileNumber', 'filmNumber', 'items', 'size', 'isXBRL',
       'isInlineXBRL', 'primaryDocument', 'primaryDocDescription'],
      dtype='object')

In [24]:
allForms[['accessionNumber', 'reportDate', 'form']].head(50)

,accessionNumber,reportDate,form
0,0001728451-23-000002,2023-03-02,4
1,0001193125-23-056259,2023-02-25,8-K
2,0000950170-23-004451,2022-12-31,10-K
3,0001728451-23-000001,2023-02-24,4
4,0001193125-23-039064,,SC 13G/A
5,0001193125-23-039005,,SC 13G
6,0000950123-23-002585,2022-12-31,13F-HR
7,0001193125-23-038392,,SC 13G
8,0001193125-23-038335,,SC 13G
9,0001193125-23-038333,,SC 13G


In [25]:
allForms[allForms['form'] == '10-K'].head()

,accessionNumber,filingDate,reportDate,acceptanceDateTime,act,form,fileNumber,filmNumber,items,size,isXBRL,isInlineXBRL,primaryDocument,primaryDocDescription
2,0000950170-23-004451,2023-02-27,2022-12-31,2023-02-27T06:04:49.000Z,34,10-K,001-14905,23669900,,54911690,1,1,brka-20221231.htm,10-K
113,0001564590-22-007322,2022-02-28,2021-12-31,2022-02-28T06:04:52.000Z,34,10-K,001-14905,22683749,,51024548,1,1,brka-10k_20211231.htm,10-K
201,0001564590-21-009611,2021-03-01,2020-12-31,2021-03-01T06:22:33.000Z,34,10-K,001-14905,21693635,,52363419,1,1,brka-10k_20201231.htm,10-K
330,0001564590-20-005874,2020-02-24,2019-12-31,2020-02-24T06:23:37.000Z,34,10-K,001-14905,20641984,,50247172,1,1,brka-10k_20191231.htm,10-K
412,0001193125-19-048926,2019-02-25,2018-12-31,2019-02-25T06:20:28.000Z,34,10-K,001-14905,19628018,,17207124,1,0,d678758d10k.htm,10-K


In [26]:
allForms.head()

,accessionNumber,filingDate,reportDate,acceptanceDateTime,act,form,fileNumber,filmNumber,items,size,isXBRL,isInlineXBRL,primaryDocument,primaryDocDescription
0,0001728451-23-000002,2023-03-03,2023-03-02,2023-03-03T21:59:09.000Z,,4,,,,8778,0,0,xslF345X03/primary_doc.xml,PRIMARY DOCUMENT
1,0001193125-23-056259,2023-03-01,2023-02-25,2023-03-01T15:57:22.000Z,34,8-K,001-14905,23693267,"2.02,9.01",320084,1,1,d305559d8k.htm,8-K
2,0000950170-23-004451,2023-02-27,2022-12-31,2023-02-27T06:04:49.000Z,34,10-K,001-14905,23669900,,54911690,1,1,brka-20221231.htm,10-K
3,0001728451-23-000001,2023-02-24,2023-02-24,2023-02-24T19:43:12.000Z,,4,,,,9267,0,0,xslF345X03/primary_doc.xml,PRIMARY DOCUMENT
4,0001193125-23-039064,2023-02-14,,2023-02-14T16:07:06.000Z,,SC 13G/A,,,,119147,0,0,d309375dsc13ga.htm,SC 13G/A


In [27]:
companyFacts = requests.get(
    f'https://data.sec.gov/api/xbrl/companyfacts/CIK{cik}.json',
    headers=headers
    )

In [28]:
companyConcept = requests.get(
    (
    f'https://data.sec.gov/api/xbrl/companyconcept/CIK{cik}'
     f'/us-gaap/Assets.json'
    ),
    headers=headers
    )

In [29]:
assetsData = pd.DataFrame.from_dict((
               companyConcept.json()['units']['USD']))

In [30]:
assetsData.head()

,end,val,accn,fy,fp,form,filed,frame
0,2008-12-31,267399000000,0001157523-09-007839,2009,Q3,10-Q,2009-11-06,NaN
1,2008-12-31,267399000000,0001193125-10-043450,2009,FY,10-K,2010-03-01,CY2008Q4I
2,2009-09-30,292010000000,0001157523-09-007839,2009,Q3,10-Q,2009-11-06,CY2009Q3I
3,2009-12-31,297119000000,0001193125-10-043450,2009,FY,10-K,2010-03-01,NaN
4,2009-12-31,297119000000,0001157523-10-002982,2010,Q1,10-Q,2010-05-07,NaN
